# Becoming a backprop ninja

This is for the 'part 4 - becoming a backprop ninja' lecture. He taked issue w/ us (i think he's feeling 'blindly') calling 'loss.backward' and so using PyTorch's autograd functionality to get our weights. He thinks it's important and useful for us to understand what's going on, as he writes about in https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b. 

We did do micrograd already, but micrograd only thinks about scalars. This lecture is about tensors.

Historically, interesting to know that back in just 2012, people wrote their backward pass by hand (or used other algorithms than backprop entirely), while now everyone just calls loss.backward(), and 'we've lost something'. They'd use Matlab! (Since it had a convenient tensor class.)

# Overall plan

We'll do the same multilayer perceptron network, and same training loop, but we'll do the backprop by hand.

# Setup

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}

vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3 # context length, how many chars to predict next char?

def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size
        # training row for each set of block_size chars (with '.' at end)
        for ch in w + '.':
            ix = stoi[ch] # 'next' char, after context
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # new context shifts right and adds prev 'next' char
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words)) # 80% for train
n2 = int(0.9*len(words)) # 10% for dev/validation and test

Xtr,  Ytr = build_dataset(words[:n1])    # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte,  Yte = build_dataset(words[n2:])    # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# new stuff

In [8]:
# util function to use later to compare manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}') 

In [9]:
n_embd = 10 # dimensionality of the character vectors
n_hidden = 64 # neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),             generator=g)
# Layer one
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # doesn't matter I think because of batch norm
# Layer two
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm params
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# note: many of the params are initialized in non-standard ways beacuse
# sometimes initializing with e.g. all zeros could mask an incorrect
# impl of the backward pass